# Biber Resultate

---

- Welche Klasse erreichte durchschnittlich die meisten/wenigsten Punkte
- Welche Aufgabe wurde am besten/schlechtesten gelöst?
- Welche Punkte wurden wie häufig gegeben?
- Wie entwicklte sich die Punktezahl der gesamten Schule über die Jahre?
- Wie entwicklte sich die Teilnehmeranzahl der gesamten Schule über die Jahre?

---

[LLM-Unterhaltung](https://chatgpt.com/share/692960a7-fc98-8009-987d-c1bea38ebd59)

Die Rohdaten sind anonymisiert, weshalb beispielsweise die Spalte "Klasse" anstelle von `G1B` oder `F2A` einen zufälligen String wie `XBE` enthält. Dies verhindert Rückschlüsse auf die Leistung einzelner Schülerinnen und Schüler oder den direkten Vergleich einzelner Klassen.

## Daten einlesen

In [ ]:
%%bash
tree

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/Biber-Daten-Long.csv")

In [ ]:
df

Wir entfernen unnötige Spalten

In [ ]:
df = df[['Jahr', 'Klassenstufe', 'Klasse', 'Name', 'Team', 'Aufgabe', 'Punkte']]

In [ ]:
df

Um uns mit den Rohdaten einfacher vertraut machen zu können, benötigen wir in Jupyter noch eine Bibliothek wie `itables`. 

In [ ]:
import itables

itables.init_notebook_mode()
itables.options.maxBytes = "1024KB"

In [ ]:
df

Zum Filtern, Durchsuchen und Exportieren der Rohdaten definieren wir noch zwei praktische Helfer-Funktionen 

In [ ]:
def filter_in(df):
    numeric_cols = df.select_dtypes(include='number').columns
    non_numeric_cols = df.select_dtypes(exclude='number').columns
    non_numeric_cols_indices = [df.columns.get_loc(c) for c in non_numeric_cols]
    itables.show(df, 
                 layout={"top1": "searchPanes"},
                 buttons = ["pageLength", "copyHtml5", "csvHtml5", "excelHtml5"],
                 searchPanes={"layout": 'columns-' + str(len(df.columns) - len(numeric_cols)), "cascadePanes": True, "columns": non_numeric_cols_indices})

def suche_in(df):
    itables.show(df,
                 layout={"top1": "searchBuilder" },
                 buttons = ["pageLength", "copyHtml5", "csvHtml5", "excelHtml5"])

In [ ]:
filter_in(df)

In [ ]:
suche_in(df)

## Daten analysieren

Einen ersten Überblick verschaffen

In [ ]:
df.describe()

In [ ]:
df.groupby("Jahr").describe()

In [ ]:
df.groupby(["Jahr", "Klasse"]).describe()

In [ ]:
df.loc[df['Jahr'] == 2025] \
  .drop(columns=['Jahr'])  \
  .groupby(["Aufgabe"])    \
  .describe()

In [ ]:
df.groupby(["Klasse","Name"]).describe()

oder erst Filtern und dann pivotisieren

In [ ]:
df.loc[df['Jahr'] == 2025].pivot_table(
    index   = ['Aufgabe'], # neue Zeilen
    columns = ['Jahr', 'Klasse'], # neue Spalten
    values  = ['Punkte'], # Werte, auf die die Aggregationsfunktionen angewendet werden
    aggfunc = ['mean','median']
)

In [ ]:
df.pivot_table(
    index   = ['Name'],
    columns = ['Aufgabe'],
    values  = ['Punkte'],
    aggfunc = ['sum'],
    margins = True,
    margins_name = 'Total'
)

### Welche Klasse erreichte durchschnittlich die meisten/wenigsten Punkte

In [ ]:
df_avg = df.groupby(["Jahr", "Klassenstufe", "Klasse"])["Punkte"].mean()

In [ ]:
df_avg.sort_values()

In [ ]:
df.boxplot(column='Punkte', by='Klasse', showmeans=True, grid=False)

### Welche Aufgabe wurde am besten/schlechtesten gelöst?

In [ ]:
df_avg = df.groupby(["Aufgabe"])["Punkte"].mean()

In [ ]:
df_avg.sort_values()

In [ ]:
df.boxplot(column="Punkte", by="Aufgabe", grid=False, showmeans=True, rot=90)

### Welche Punkte wurden wie häufig gegeben?

In [ ]:
df[["Punkte"]].hist()

### Wie entwickelt sich die durchschnittliche Punktezahl der Schule über die Jahre?

In [ ]:
df_avg = (
    df
    .groupby(["Jahr"])["Punkte"]
    .mean()
)

In [ ]:
df_avg.sort_values()

In [ ]:
df.boxplot(column='Punkte', by='Jahr', showmeans=True, grid=False)

### Wie entwicklte sich die Teilnehmeranzahl der gesamten Schule über die Jahre?

In [ ]:
participants_per_year = (
    df.groupby("Jahr")["Name"]
      .nunique()
)

In [ ]:
participants_per_year

In [ ]:
participants_per_year.plot.bar()

## Anhang

Bilder für Präsentationen und Dokumente sopeicherst du deispielsweise so:

In [ ]:
img = df.boxplot(column='Punkte', by='Jahr', showmeans=True, grid=False)
img.get_figure().savefig('img/a_boxplot.svg')
img.get_figure().savefig('img/a_boxplot.png')